In [1]:
#Maxwellsches fallrad
import numpy as np
import scipy as sp
import sympy as smp
from IPython.display import display, Math, Latex
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.axes as axes
import math
import statistics as stat
from scipy import optimize
import uncertainties as uc
import uncertainties.unumpy as unp 
unv=unp.nominal_values
usd=unp.std_devs

In [48]:
tp=1.14
#Volumen rad best J
#Dicke Rad D->H_k
D=10**(-3)*np.array([11.68,11.56,11.60,11.72,11.54])
D_m=np.mean(D)
uD1=np.sqrt((1e-3*0.06/np.sqrt(6))**2)
uD2=tp*np.sqrt(np.dot(D-D_m,D-D_m)/(len(D)*(len(D)-1)))
uD=np.sqrt(uD1**2+uD2**2)
H_k=D_m
uH_k=uD
H_k,uH_k

(0.01162, 4.6470635889774607e-05)

In [49]:
#Radius außen R_a
R_aroh=0.5*10**(-3)*np.array([170.18,170.20,170.40,170.14,170.22])
R_a=np.mean(R_aroh)
uR_a=np.sqrt((1e-3*0.06/np.sqrt(6))**2+tp*np.dot(R_aroh-R_a,R_aroh-R_a)/(len(R_aroh)*(len(R_aroh)-1)))
R_a,uR_a

(0.085113999999999995, 3.430510166141509e-05)

In [50]:
#Radius innen R_i
R_iroh=0.5*10**(-3)*np.array([145.40,145.48,145.20,145.56,145.50])
R_i=np.mean(R_iroh)
uR_i=np.sqrt((1e-3*0.06/np.sqrt(6))**2+tp*np.dot(R_iroh-R_i,R_iroh-R_i)/(len(R_iroh)*(len(R_iroh)-1)))
H_S=R_i
uH_S=uR_i
R_i,uR_i

(0.072714000000000001, 4.138405490040802e-05)

In [51]:
#Radius Speiche- speichen gleich dick -> mittelung
R_Sroh=0.5*10**(-3)*np.array([8.04,8.06,8.04,8.1])
R_S=np.mean(R_Sroh)
uR_S=np.sqrt((1e-3*0.06/np.sqrt(6))**2+tp*np.dot(R_Sroh-R_S,R_Sroh-R_S)/(len(R_Sroh)*(len(R_Sroh)-1)))
R_S,uR_S

(0.0040299999999999997, 2.5632011235952637e-05)

In [52]:
#Höhe Achse
H_A=200.3e-3
uH_A=1e-3*0.06/np.sqrt(6)
#Masse
M=0.73204
uM=1e-3*0.01/np.sqrt(3)
#Radius Achse
R_Aroh=0.5e-3*np.array([8.1,8.12,8.12,8.08,8.08])
R_A=np.mean(R_Aroh)
uR_A=np.sqrt((1e-3*0.06/np.sqrt(6))**2+tp*np.dot(R_Aroh-R_A,R_Aroh-R_A)/(len(R_Aroh)*(len(R_Aroh)-1)))
M,uM,H_A,uH_A,R_A,uR_A

(0.73204,
 5.7735026918962587e-06,
 0.2003,
 2.4494897427831785e-05,
 0.0040499999999999998,
 2.4955961211702471e-05)

In [53]:
#Hier die Variablen deklarieren/ "d" ist reserviert und darf nicht verwendet werden
vM,vH_k,vR_a,vR_i,vH_S,vR_S,vH_A,vR_A = smp.symbols('M H_k R_a R_i H_S R_S H_A R_A') #hier Eure Variablennamen einsetzen
variablen = [vM, vH_k,vR_a,vR_i,vH_S,vR_S,vH_A,vR_A]         #hier Eure Variablennamen einsetzen
#Zahlenwerte und Fehlerwerte
variablen_werte = [M,H_k,R_a,R_i,R_i,R_S,H_A,R_A]  #Werte h=2.8, r=4.2, T=2.4 /hier Eure Werte einsetzen
fehler_werte = [uM,uH_k,uR_a,uR_i,uR_i,uR_S,uH_A,uR_A]   # dito fuer die Fehler /hier Eure Werte einsetzen

In [54]:
#funktion definieren hier das Trägheitsmoment
#vol: funktion = vH_k*smp.pi*(vR_a**2-vR_i**2)+2*vR_i*smp.pi*vR_S**2+vH_A*smp.pi*vR_A**2
#funktion=M/(H_k*(R_a**2-R_i**2)+2*R_i*R_S**2+H_A*R_A**2)
funktion=vM/(vH_k*(vR_a**2-vR_i**2)+2*vR_i*vR_S**2+vH_A*vR_A**2)*\
(0.5*vH_k*(vR_a**4-vR_i**4)+2*vH_S*(vH_S**2*vR_S**2/12+vR_S**4/4+vH_A*vR_A**4/2))
print(smp.latex((funktion)))

\frac{M \left(2 H_{S} \left(\frac{H_{A} R_{A}^{4}}{2} + \frac{H_{S}^{2} R_{S}^{2}}{12} + \frac{R_{S}^{4}}{4}\right) + 0.5 H_{k} \left(R_{a}^{4} - R_{i}^{4}\right)\right)}{H_{A} R_{A}^{2} + H_{k} \left(R_{a}^{2} - R_{i}^{2}\right) + 2 R_{S}^{2} R_{i}}


In [9]:
fehler = 0
fehlersymbole=[]
ableitungen_quadr = []

for var in variablen:
    d = smp.symbols('d' + var.name)        #Symbole fuer die Fehler generieren
    fehlersymbole.append(d)               #Fehlersymbole in Liste eintragen
    partial = smp.diff(funktion, var) * d  #Partielle Differentation und mit mit Fehlersymbol 'd' multiplizieren
    ableitungen_quadr.append(partial**2)  
    fehler = fehler + partial**2

fehler_abs=smp.simplify(smp.sqrt(fehler))              #Latex Format fuer den absoluten Fehler
fehler_rel=smp.simplify(smp.sqrt(fehler/funktion**2))  #Latex Format fuer den relativen Fehler
print(fehler_abs)

sqrt((4*M**2*R_S**2*dR_S**2*(H_S*(H_S**2 + 6*R_S**2)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - 2*R_i*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4)))**2 + M**2*(4*H_A**2*R_A**2*dR_A**2*(12*H_S*R_A**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4))**2 + 4*H_k**2*R_a**2*dR_a**2*(-H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4) + 6.0*R_a**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i))**2 + R_A**4*dH_A**2*(6*H_S*R_A**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4))**2 + dH_k**2*((R_a**2 - R_i**2)*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4)) - 3.0*(R_a**4 - R_i**4)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i))**2 + dR_i**2*(12.0*H_k*R_i**3*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - 2*(H_S*(6*H_A*R_A**4 + H_S**

In [10]:
#Berechnung der Zahlenwerte
funktions_wert=smp.Subs(funktion,variablen,variablen_werte).doit() #Variablenwerte (Zahlen) in Formel einsetzen und
                                                                  #Funktionswert berechnen
err1=smp.Subs(fehler,variablen,variablen_werte).doit()             #Variablenwerte (Zahlen) in FehlerFormel einsetzen 

err2=smp.Subs(err1,fehlersymbole,fehler_werte).doit()              #Variablenwerte (Zahlen) in FehlerFormel einsetzen und
                                                                  #Funktionswert berechnen
print(err2)

6.36452170087612e-11


In [11]:
#Latex Darstellungen
print('Funktion:')
display(Math("f="+smp.latex(funktion)))

print('Messwerte:')
for i in range(len(variablen)):
    display(Math(str(variablen[i])+'='+ str(variablen_werte[i])+'\pm '+ str(fehler_werte[i])))  #Messwerte mit Fehler

print('Absoluter Fehler:')
display(Math(r'\Delta f='+smp.latex(fehler_abs).replace('d',r'\Delta ')))   #Formel absoluter Fehler
print('Relativer Fehler:')
display(Math(r"\Delta f/f="+smp.latex(fehler_rel).replace('d',r'\Delta '))) #Formel relativer Fehler
display(Math("f= %6.2f \pm %6.2f" %(funktions_wert,smp.sqrt(err2))))        #Messwert und Wert des absoluten Fehler
display(Math("f= %6.2f \pm %6.1f %s" %(funktions_wert,smp.sqrt(err2)/funktions_wert*100," \%"))) #dito als relativer Fehler

Funktion:


<IPython.core.display.Math object>

Messwerte:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

Relativer Fehler:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
#Latex code
print('Funktion:')
display(smp.latex("V=f="+smp.latex(funktion)))

print('Messwerte:')
for i in range(len(variablen)):
    display(smp.latex(str(variablen[i])+'='+ str(variablen_werte[i])+'\pm '+ str(fehler_werte[i])))  #Messwerte mit Fehler

print('Absoluter Fehler:')
display(smp.latex(r'\Delta f='+smp.latex(fehler_abs).replace('d',r'\Delta ')))   #Formel absoluter Fehler
print('Relativer Fehler:')
display(smp.latex(r"\Delta f/f="+smp.latex(fehler_rel).replace('d',r'\Delta '))) #Formel relativer Fehler
display(smp.latex("f= %6.2f \pm %6.2f" %(funktions_wert,smp.sqrt(err2))))        #Messwert und Wert des absoluten Fehler
smp.latex(smp.latex("f= %6.2f \pm %6.1f %s" %(funktions_wert,smp.sqrt(err2)/funktions_wert*100," \%"))) #dito als relativer Fehler
#der code ist noch nicht ganz so toll es muss noch \\->\ und $$

Funktion:


'V=f=\\frac{M \\left(2 H_{S} \\left(\\frac{H_{A} R_{A}^{4}}{2} + \\frac{H_{S}^{2} R_{S}^{2}}{12} + \\frac{R_{S}^{4}}{4}\\right) + 0.5 H_{k} \\left(R_{a}^{4} - R_{i}^{4}\\right)\\right)}{H_{A} R_{A}^{2} + H_{k} \\left(R_{a}^{2} - R_{i}^{2}\\right) + 2 R_{S}^{2} R_{i}}'

Messwerte:


'M=0.73204\\pm 5.7735026919e-06'

'H_k=0.01162\\pm 4.64706358898e-05'

...

'\\Delta f=\\frac{1}{6} \\sqrt{\\frac{1}{\\left(H_{A} R_{A}^{2} + H_{k} \\left(R_{a}^{2} - R_{i}^{2}\\right) + 2 R_{S}^{2} R_{i}\\right)^{4}} \\left(4 M^{2} R_{S}^{2} \\Delta R_{S}^{2} \\left(H_{S} \\left(H_{S}^{2} + 6 R_{S}^{2}\\right) \\left(H_{A} R_{A}^{2} + H_{k} \\left(R_{a}^{2} - R_{i}^{2}\\right) + 2 R_{S}^{2} R_{i}\\right) - 2 R_{i} \\left(H_{S} \\left(6 H_{A} R_{A}^{4} + H_{S}^{2} R_{S}^{2} + 3 R_{S}^{4}\\right) + 3.0 H_{k} \\left(R_{a}^{4} - R_{i}^{4}\\right)\\right)\\right)^{2} + M^{2} \\left(4 H_{A}^{2} R_{A}^{2} \\Delta R_{A}^{2} \\left(12 H_{S} R_{A}^{2} \\left(H_{A} R_{A}^{2} + H_{k} \\left(R_{a}^{2} - R_{i}^{2}\\right) + 2 R_{S}^{2} R_{i}\\right) - H_{S} \\left(6 H_{A} R_{A}^{4} + H_{S}^{2} R_{S}^{2} + 3 R_{S}^{4}\\right) - 3.0 H_{k} \\left(R_{a}^{4} - R_{i}^{4}\\right)\\right)^{2} + 4 H_{k}^{2} R_{a}^{2} \\Delta R_{a}^{2} \\left(- H_{S} \\left(6 H_{A} R_{A}^{4} + H_{S}^{2} R_{S}^{2} + 3 R_{S}^{4}\\right) - 3.0 H_{k} \\left(R_{a}^{4} - R_{i}^{4}\\right) + 6.0 R_{a}^{2} 

Relativer Fehler:


'\\Delta f/f=\\sqrt{\\frac{1}{M^{2} \\left(H_{S} \\left(6 H_{A} R_{A}^{4} + H_{S}^{2} R_{S}^{2} + 3 R_{S}^{4}\\right) + 3.0 H_{k} \\left(R_{a}^{4} - R_{i}^{4}\\right)\\right)^{2} \\left(H_{A} R_{A}^{2} + H_{k} \\left(R_{a}^{2} - R_{i}^{2}\\right) + 2 R_{S}^{2} R_{i}\\right)^{2}} \\left(4 M^{2} R_{S}^{2} \\Delta R_{S}^{2} \\left(H_{S} \\left(H_{S}^{2} + 6 R_{S}^{2}\\right) \\left(H_{A} R_{A}^{2} + H_{k} \\left(R_{a}^{2} - R_{i}^{2}\\right) + 2 R_{S}^{2} R_{i}\\right) - 2 R_{i} \\left(H_{S} \\left(6 H_{A} R_{A}^{4} + H_{S}^{2} R_{S}^{2} + 3 R_{S}^{4}\\right) + 3.0 H_{k} \\left(R_{a}^{4} - R_{i}^{4}\\right)\\right)\\right)^{2} + M^{2} \\left(4 H_{A}^{2} R_{A}^{2} \\Delta R_{A}^{2} \\left(12 H_{S} R_{A}^{2} \\left(H_{A} R_{A}^{2} + H_{k} \\left(R_{a}^{2} - R_{i}^{2}\\right) + 2 R_{S}^{2} R_{i}\\right) - H_{S} \\left(6 H_{A} R_{A}^{4} + H_{S}^{2} R_{S}^{2} + 3 R_{S}^{4}\\right) - 3.0 H_{k} \\left(R_{a}^{4} - R_{i}^{4}\\right)\\right)^{2} + 4 H_{k}^{2} R_{a}^{2} \\Delta R_{a}^{2} \\left(- H_

'f=   0.00 \\pm   0.00'

'f=   0.00 \\pm    0.2  \\%'

In [55]:

uJ='np.sqrt((4*M**2*R_S**2*dR_S**2*(H_S*(H_S**2 + 6*R_S**2)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - 2*R_i*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4)))**2 + M**2*(4*H_A**2*R_A**2*dR_A**2*(12*H_S*R_A**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4))**2 + 4*H_k**2*R_a**2*dR_a**2*(-H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4) + 6.0*R_a**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i))**2 + R_A**4*dH_A**2*(6*H_S*R_A**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4))**2 + dH_k**2*((R_a**2 - R_i**2)*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4)) - 3.0*(R_a**4 - R_i**4)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i))**2 + dR_i**2*(12.0*H_k*R_i**3*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - 2*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4))*(H_k*R_i - R_S**2))**2) + (9*M**2*dH_S**2*(2*H_A*R_A**4 + H_S**2*R_S**2 + R_S**4)**2 + dM**2*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4))**2)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i)**2)/(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i)**4)/6'
uJ.replace('d','u')


'np.sqrt((4*M**2*R_S**2*uR_S**2*(H_S*(H_S**2 + 6*R_S**2)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - 2*R_i*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4)))**2 + M**2*(4*H_A**2*R_A**2*uR_A**2*(12*H_S*R_A**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4))**2 + 4*H_k**2*R_a**2*uR_a**2*(-H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4) + 6.0*R_a**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i))**2 + R_A**4*uH_A**2*(6*H_S*R_A**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4))**2 + uH_k**2*((R_a**2 - R_i**2)*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4)) - 3.0*(R_a**4 - R_i**4)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i))**2 + uR_i**2*(12.0*H_k*R_i**3*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - 2*(H_S*(6*H_A*R_A**4 + H

In [56]:
uJ=np.sqrt((4*M**2*R_S**2*uR_S**2*(H_S*(H_S**2 + 6*R_S**2)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - 2*R_i*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4)))**2 + M**2*(4*H_A**2*R_A**2*uR_A**2*(12*H_S*R_A**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4))**2 + 4*H_k**2*R_a**2*uR_a**2*(-H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4) + 6.0*R_a**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i))**2 + R_A**4*uH_A**2*(6*H_S*R_A**2*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) - 3.0*H_k*(R_a**4 - R_i**4))**2 + uH_k**2*((R_a**2 - R_i**2)*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4)) - 3.0*(R_a**4 - R_i**4)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i))**2 + uR_i**2*(12.0*H_k*R_i**3*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i) - 2*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4))*(H_k*R_i - R_S**2))**2) + (9*M**2*uH_S**2*(2*H_A*R_A**4 + H_S**2*R_S**2 + R_S**4)**2 + uM**2*(H_S*(6*H_A*R_A**4 + H_S**2*R_S**2 + 3*R_S**4) + 3.0*H_k*(R_a**4 - R_i**4))**2)*(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i)**2)/(H_A*R_A**2 + H_k*(R_a**2 - R_i**2) + 2*R_S**2*R_i)**4)/6
uJ,7.8188769594473602e-06

(7.9777952473575914e-06, 7.81887695944736e-06)

In [57]:
J=M/(H_k*(R_a**2-R_i**2)+2*R_i*R_S**2+H_A*R_A**2)*\
(0.5*H_k*(R_a**4-R_i**4)+2*H_S*(H_S**2*R_S**2/12+R_S**4/4+H_A*R_A**4/2))
J

0.0037015804223111274

In [58]:
uJ/J, uJ*100/J,(0.0021123077354525091, 0.21123077354525094)

(0.0021552402858173096,
 0.21552402858173095,
 (0.002112307735452509, 0.21123077354525094))

In [59]:
#Kontrolle Dichte
rho=M/(H_k*np.pi*(R_a**2-R_i**2)+2*R_i*np.pi*R_S**2+H_A*np.pi*R_A**2)
rho

8208.1194606252866

In [60]:
#Vergleich dichte Eisen
(rho-7874)/rho 

0.04070596952542814

In [61]:
#Radienvergleich c kopiert aus anderem notebook
c=0.020482267595833874
uc=0.00093070450146481875
g=9.81

In [62]:
R1=np.sqrt(2*c*J/(M*(g-2*c)))
uR1=np.sqrt(((uM*c*J)/(np.sqrt(2)*(-2*c+g)*np.sqrt((c*J)/((-2*c+g)*M))*M**2))**2\
+(uJ*c/(np.sqrt(2)*(-2*c+g)*np.sqrt((J*c)/((-2*c+g)*M))*M))**2+(uc*g*np.sqrt(J*c)*(np.sqrt(2*g*M-4*M*c)*c*(g-2*c))**(-1))**2)
#uR1=R1*np.sqrt((uM/(2*M))**2+(uJ/(2*J)**2)+(uc/c)**2) #Funktioniert aber falsch?
R1,uR1,(0.0046047320243258307, 0.00010516962121265645)

(0.0046047320243258307,
 0.00010517423846911241,
 (0.004604732024325831, 0.00010516962121265645))

In [65]:
#Faden:
R_Froh=0.5*10**(-3)*np.array([0.98,1.02,0.98,1.02,1.00])
R_F=np.mean(R_Froh)
uR_F=np.sqrt((1e-3*0.06/np.sqrt(6))**2+tp*np.dot(R_Froh-R_F,R_Froh-R_F)/(len(R_Froh)*(len(R_Froh)-1)))
#Radius gemessen
R2=R_A+R_F
uR2=np.sqrt(uR_a**2+uR_F**2)
R_F,uR_F,R2,uR2#(0.00050000000000000001, 2.4899799195977473e-05, 0.0045500000000000002, 4.1545156155682233e-05)

(0.00050000000000000001,
 2.4955961211702512e-05,
 0.0045500000000000002,
 4.2422164018352775e-05)

In [3]:
uD1=np.sqrt((1e-3*0.06/np.sqrt(6))**2)
uD1*1000

0.024494897427831785